In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pandas_ta as ta
import numpy as np
import requests
from bs4 import BeautifulSoup

# Function to fetch S&P 500 tickers from Wikipedia
# def get_sp500_tickers():
#     url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     table = soup.find('table', {'id': 'constituents'})
#     tickers = [row.find('td').text.strip() for row in table.find_all('tr')[1:]]
#     return tickers

# # Get the S&P 500 tickers
# sp500_tickers = get_sp500_tickers()

# # Save the tickers to a CSV file
# pd.DataFrame(sp500_tickers, columns=["Ticker"]).to_csv('sp500_tickers.csv', index=False)


In [14]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Function to fetch S&P 500 tickers from Wikipedia
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = [row.find('td').text.strip() for row in table.find_all('tr')[1:]]
    return tickers# Get only the first 10 tickers

# Get the S&P 500 tickers
sp500_tickers = get_sp500_tickers()

# Display the tickers
print("S&P 500 Tickers:")
print(sp500_tickers)

# Save the tickers to a CSV file
pd.DataFrame(sp500_tickers, columns=["Ticker"]).to_csv('sp500_tickers.csv', index=False)


S&P 500 Tickers:
['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK.B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BWA', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 

In [5]:

# Define the timeframes and their corresponding date ranges

# Read the tickers from the CSV file
tickers_df = pd.read_csv('sp500_tickers.csv')
tickers = tickers_df['Ticker'].tolist()

timeframes = {
    # '15m': timedelta(days=60),
    # '1h': timedelta(days=730),
    '1d': timedelta(days=180),
    # '1wk': timedelta(days=3*365)
}

# Define the recent period (in days)
recent_period = 1

# Function to download data for a given ticker and timeframe
def download_data(ticker, timeframe, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date, interval=timeframe)

# Dictionary to store data for each timeframe
data = {tf: {} for tf in timeframes}

# Download data for each ticker and timeframe
end_date = datetime.now()
for ticker in tickers:
    for tf, delta in timeframes.items():
        start_date = end_date - delta
        try:
            data[tf][ticker] = download_data(ticker, tf, start_date, end_date)
        except Exception as e:
            print(f"Failed to download data for {ticker} with timeframe {tf}: {e}")


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['MMM']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AOS']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


KeyboardInterrupt: 

In [8]:


# List to store screener results
screener_results = []

# Applying the custom indicator and generating buy/sell signals
for tf in timeframes:
    for ticker in data[tf]:
        df = data[tf][ticker]
        
        # Round OHLC values to two decimal points
        df['Open'] = df['Open'].round(2)
        df['High'] = df['High'].round(2)
        df['Low'] = df['Low'].round(2)
        df['Close'] = df['Close'].round(2)
        
       # --- Existing calculations ---
        # Linear Regression Curves
        df['reg1'] = ta.linreg(df['Close'], length=10)
        df['reg2'] = ta.linreg(df['Close'], length=14)
        df['reg3'] = ta.linreg(df['Close'], length=30)

        # R-squared Calculation
        r2_length = 14
        df['r2_raw'] = df['Close'].rolling(window=r2_length).apply(
            lambda x: np.corrcoef(x, np.arange(r2_length))[0, 1]**2
        )
        df['r2'] = df['r2_raw'] * 100  # Normalized to [0, 100]
        df['r2_smoothed'] = df['r2'].rolling(window=3).mean()

        # --- Flatten the columns if they are a MultiIndex ---
        if isinstance(df.columns, pd.MultiIndex):
            # This will take the first level of the MultiIndex as the new columns.
            df.columns = df.columns.get_level_values(0)

        # --- RSI Calculation using the DataFrame accessor ---
        # Explicitly specify the 'Close' column.
        df.ta.rsi(close='Close', length=14, append=True)
        # This will add a new column named 'RSI_14' to df.

        # --- Buy and Sell Signals ---
        df['buy_signal'] = np.where((df['r2_smoothed'] > 90) & (df['RSI_14'] < 30), 1, 0)
        df['sell_signal'] = np.where((df['r2_smoothed'] > 90) & (df['RSI_14'] > 70), 1, 0)


                
                # Save the DataFrame to a CSV file
        df.to_csv(f'{ticker}_{tf}_data.csv')
        
        
        # Filter for recent periods
        recent_date_cutoff = df.index.max() - pd.Timedelta(days=recent_period)
        df = df[df.index >= recent_date_cutoff]
        
        # Filter rows with buy or sell signals
        df_filtered = df[(df['buy_signal'] == 1) | (df['sell_signal'] == 1)]
        
        # Append results to screener list
        if not df_filtered.empty:
            for index, row in df_filtered.iterrows():
                screener_results.append({
                    'Ticker': ticker,
                    'Date': index,
                    'Buy Signal': row['buy_signal'],
                    'Sell Signal': row['sell_signal']
                })

# Save screener results to a CSV file
screener_df = pd.DataFrame(screener_results)
# screener_df.to_csv('screener_results.csv', index=False)

# Display a sample of the screener results
print("Screener results:")
print(screener_df)


Screener results:
  Ticker       Date  Buy Signal  Sell Signal
0    MMM 2025-01-30         0.0          1.0
1    MMM 2025-01-31         0.0          1.0


# regession line crossed ta alerts

In [4]:

# Read the tickers from the CSV file
tickers_df = pd.read_csv('sp500_tickers.csv')
tickers = tickers_df['Ticker'].tolist()

# Define the timeframes and their corresponding date ranges
timeframes = {
    # '15m': timedelta(days=60),
    '1h': timedelta(days=30),
    # '1d': timedelta(days=3*365),
    # '1wk': timedelta(days=3*365)
}

# Define the recent period (in days)
recent_period = 1

# Function to download data for a given ticker and timeframe
def download_data(ticker, timeframe, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date, interval=timeframe)

# Dictionary to store data for each timeframe
data = {tf: {} for tf in timeframes}

# Download data for each ticker and timeframe
end_date = datetime.now()
for ticker in tickers:
    for tf, delta in timeframes.items():
        start_date = end_date - delta
        try:
            data[tf][ticker] = download_data(ticker, tf, start_date, end_date)
        except Exception as e:
            print(f"Failed to download data for {ticker} with timeframe {tf}: {e}")


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['MMM']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AOS']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ABT']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ABBV']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ACN']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ADBE']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%******************

KeyboardInterrupt: 

In [5]:

# List to store screener results
screener_results = []

# Applying the custom indicator and generating buy/sell signals
for tf in timeframes:
    for ticker in data[tf]:
        df = data[tf][ticker]
        
                # If the DataFrame columns are a MultiIndex, flatten them
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        
        # Round OHLC values to two decimal points
        df['Open'] = df['Open'].round(2)
        df['High'] = df['High'].round(2)
        df['Low'] = df['Low'].round(2)
        df['Close'] = df['Close'].round(2)
        
       # --- Existing calculations ---
        # Linear Regression Curves
        
        df['reg1'] = ta.linreg(df['Close'], length=25)
        df['reg2'] = ta.linreg(df['Close'], length=50)
        
        
        # df['reg3'] = ta.linreg(df['Close'], length=30)

        # R-squared Calculation
        # r2_length = 14
        # df['r2_raw'] = df['Close'].rolling(window=r2_length).apply(
        #     lambda x: np.corrcoef(x, np.arange(r2_length))[0, 1]**2
        # )
        # df['r2'] = df['r2_raw'] * 100  # Normalized to [0, 100]
        # df['r2_smoothed'] = df['r2'].rolling(window=3).mean()

        # --- Flatten the columns if they are a MultiIndex ---
        # if isinstance(df.columns, pd.MultiIndex):
        #     # This will take the first level of the MultiIndex as the new columns.
        #     df.columns = df.columns.get_level_values(0)

        # --- RSI Calculation using the DataFrame accessor ---
        # Explicitly specify the 'Close' column.
        # df.ta.rsi(close='Close', length=14, append=True)
        # This will add a new column named 'RSI_14' to df.

        # --- Buy and Sell Signals ---
        # df['buy_signal'] = np.where((df['r2_smoothed'] > 90) & (df['RSI_14'] < 30), 1, 0)
        # df['sell_signal'] = np.where((df['r2_smoothed'] > 90) & (df['RSI_14'] > 70), 1, 0)

        df['buy_signal'] = np.where(
            (df['reg1'] > df['reg2']) & (df['reg1'].shift(1) <= df['reg2'].shift(1)),
            1,
            0
        )
        
        # Sell Signal: When the 25-period line (reg1) crosses below the 50-period line (reg2)
        df['sell_signal'] = np.where(
            (df['reg1'] < df['reg2']) & (df['reg1'].shift(1) >= df['reg2'].shift(1)),
            1,
            0
        )
                
                # Save the DataFrame to a CSV file
        df.to_csv(f'{ticker}_{tf}_data.csv')
        
        
        # Filter for recent periods
        recent_date_cutoff = df.index.max() - pd.Timedelta(days=recent_period)
        df = df[df.index >= recent_date_cutoff]
        
        # Filter rows with buy or sell signals
        df_filtered = df[(df['buy_signal'] == 1) | (df['sell_signal'] == 1)]
        
        # Append results to screener list
        if not df_filtered.empty:
            for index, row in df_filtered.iterrows():
                screener_results.append({
                    'Ticker': ticker,
                    'Date': index,
                    'Buy Signal': row['buy_signal'],
                    'Sell Signal': row['sell_signal']
                })

# Save screener results to a CSV file
screener_df = pd.DataFrame(screener_results)
# screener_df.to_csv('screener_results.csv', index=False)

# Display a sample of the screener results
print("Screener results:")
print(screener_df)


Screener results:
  Ticker                      Date  Buy Signal  Sell Signal
0   ABBV 2025-01-31 16:30:00+00:00         1.0          0.0
1   ADBE 2025-01-31 15:30:00+00:00         0.0          1.0


# test previous code

In [7]:
import pytz
from datetime import datetime, timedelta




    
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pandas_ta as ta
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv
load_dotenv()


# Function to fetch S&P 500 tickers from Wikipedia
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = [row.find('td').text.strip() for row in table.find_all('tr')[1:]]
    return tickers

# Function to send message to Telegram
def send_telegram_message(message):
    bot_token = os.getenv("TELEGRAM_BOT_TOKEN")
    chat_id = os.getenv("TELEGRAM_CHAT_ID")
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    data = {"chat_id": chat_id, "text": message}
    requests.post(url, data=data)

    # Get the S&P 500 tickers
sp500_tickers = get_sp500_tickers()

# Ensure the stockdata directory exists
os.makedirs('stockdata', exist_ok=True)

# Save the tickers to a CSV file in the stockdata folder
pd.DataFrame(sp500_tickers, columns=["Ticker"]).to_csv('stockdata/sp500_tickers.csv', index=False)

# Read the tickers from the CSV file in the stockdata folder
tickers_df = pd.read_csv('stockdata/sp500_tickers.csv')
tickers = tickers_df['Ticker'].tolist()

# Define the timeframes and their corresponding date ranges
timeframes = {
    # '15m': timedelta(days=60),
    '1h': timedelta(days=30),
    # '1d': timedelta(days=90),
    # '1wk': timedelta(days=3*365)
}

# Define the recent period (in days)
recent_period = 1

# Function to download data for a given ticker and timeframe
def download_data(ticker, timeframe, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date, interval=timeframe)

# Dictionary to store data for each timeframe
data = {tf: {} for tf in timeframes}

# Download historical data for each ticker and timeframe
end_date = datetime.now()
for ticker in tickers:
    for tf, delta in timeframes.items():
        start_date = end_date - delta
        try:
            df = download_data(ticker, tf, start_date, end_date)
            if df.empty:
                print(f"No data available for {ticker} on timeframe {tf}. Skipping.")
                continue
            data[tf][ticker] = df
        except Exception as e:
            print(f"Failed to download data for {ticker} with timeframe {tf}: {e}")



# List to store screener results
screener_results = []

# Applying the custom indicator and generating buy/sell signals
for tf in timeframes:
    for ticker in data[tf]:
        df = data[tf][ticker]
        
                # If the DataFrame columns are a MultiIndex, flatten them
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        
        # Round OHLC values to two decimal points
        df['Open'] = df['Open'].round(2)
        df['High'] = df['High'].round(2)
        df['Low'] = df['Low'].round(2)
        df['Close'] = df['Close'].round(2)
        
    # --- Existing calculations ---
        # Linear Regression Curves
        
        df['reg1'] = ta.linreg(df['Close'], length=25)
        df['reg2'] = ta.linreg(df['Close'], length=50)
        

        df['buy_signal'] = np.where(
            (df['reg1'] > df['reg2']) & (df['reg1'].shift(1) <= df['reg2'].shift(1)),
            1,
            0
        )
        
        # Sell Signal: When the 25-period line (reg1) crosses below the 50-period line (reg2)
        df['sell_signal'] = np.where(
            (df['reg1'] < df['reg2']) & (df['reg1'].shift(1) >= df['reg2'].shift(1)),
            1,
            0
        )
                
        # Save each ticker's data to a CSV file in the stockdata folder
        df.to_csv(f'stockdata/{ticker}_{tf}_data.csv')

        
        hoursback =1 
        # Filter for recent periods
        recent_date_cutoff = df.index.max() - pd.Timedelta(hours=hoursback)
        df = df[df.index >= recent_date_cutoff]
        
        # Filter rows with buy or sell signals
        df_filtered = df[(df['buy_signal'] == 1) | (df['sell_signal'] == 1)]
        
        # Append results to screener list
        if not df_filtered.empty:
            for index, row in df_filtered.iterrows():
                screener_results.append({
                    'Ticker': ticker,
                    'Date': index,
                    'Buy Signal': row['buy_signal'],
                    'Sell Signal': row['sell_signal']
                })

# Save screener results to a CSV file
screener_df = pd.DataFrame(screener_results)
screener_df.to_csv('Regression_cross_screener_results_1h.csv', index=False)


# Send results to Telegram
if not screener_df.empty:
    message = "Hourly screener Results:\n Buy=linreg 25 crossover linreg 50 \n Sell=linreg 25 cross below linreg 50 \n" + screener_df.to_string(index=False)
    send_telegram_message(message)

# Display a sample of the screener results
print("Hourly screener Results:\n Buy=linreg 25 crossover linreg 50 \n Sell=linreg 25 cross below linreg 50 \n")
print(screener_df.tail())


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['MMM']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


No data available for MMM on timeframe 1h. Skipping.


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AOS']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


No data available for AOS on timeframe 1h. Skipping.


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ABT']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


...

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ABBV']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


No data available for ABBV on timeframe 1h. Skipping.


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ACN']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


No data available for ACN on timeframe 1h. Skipping.


KeyboardInterrupt: 

In [15]:
print(sp500_tickers)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK.B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BWA', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', 'DRI', 'DV

In [12]:
import requests_cache
session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'my-program/1.0'
ticker = yf.Ticker('AAPL', session=session)

# The scraped response will be stored in the cache
ticker.actions

,Dividends,Stock Splits
Date,,
1987-05-11 00:00:00-04:00,0.000536,0.0
1987-06-16 00:00:00-04:00,0.000000,2.0
1987-08-10 00:00:00-04:00,0.000536,0.0
1987-11-17 00:00:00-05:00,0.000714,0.0
1988-02-12 00:00:00-05:00,0.000714,0.0
...,...,...
2024-02-09 00:00:00-05:00,0.240000,0.0
2024-05-10 00:00:00-04:00,0.250000,0.0
2024-08-12 00:00:00-04:00,0.250000,0.0


In [14]:
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
   pass

session = CachedLimiterSession(
   limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
   bucket_class=MemoryQueueBucket,
   backend=SQLiteCache("yfinance.cache"),
)

In [ ]:
import yfinance as yf
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
   pass

session = CachedLimiterSession(
   limiter=Limiter(RequestRate(2, Duration.SECOND*5)),  # max 2 requests per 5 seconds
   bucket_class=MemoryQueueBucket,
   backend=SQLiteCache("yfinance.cache"),
)
dat = yf.Tickers(sp500_tickers,session=session) 

# get historical market data
his_data = dat.history(period='1mo', interval='60m')
# Print the columns of the downloaded historical data
print(his_data.columns)


[***********************************159%*************************************]  802 of 503 completed

[*********************100%***********************]  503 of 503 completed
2025-02-23 18:29:38,543 ERROR: 
3 Failed downloads:
2025-02-23 18:29:38,544 ERROR: ['BRK.B']: YFPricesMissingError('possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")')
2025-02-23 18:29:38,549 ERROR: ['ETR', 'BF.B']: YFPricesMissingError('possibly delisted; no price data found  (period=1mo)')


MultiIndex([('Adj Close',  'BF.B'),
            ('Adj Close', 'BRK.B'),
            ('Adj Close',   'ETR'),
            (    'Close',     'A'),
            (    'Close',  'AAPL'),
            (    'Close',  'ABBV'),
            (    'Close',  'ABNB'),
            (    'Close',   'ABT'),
            (    'Close',  'ACGL'),
            (    'Close',   'ACN'),
            ...
            (   'Volume',   'WTW'),
            (   'Volume',    'WY'),
            (   'Volume',  'WYNN'),
            (   'Volume',   'XEL'),
            (   'Volume',   'XOM'),
            (   'Volume',   'XYL'),
            (   'Volume',   'YUM'),
            (   'Volume',   'ZBH'),
            (   'Volume',  'ZBRA'),
            (   'Volume',   'ZTS')],
           names=['Price', 'Ticker'], length=3518)


[******************************************************************************337%********************************************************************************]  1695 of 503 completed2025-02-23 18:32:01,870 ERROR: Unable to deserialize response: a bytes-like object is required, not 'NoneType'
[**********************************************************************************************407%*************************************************************************************************]  2049 of 503 completed

In [ ]:
import os
import time
import pytz
import requests
import pandas as pd
import numpy as np
import pandas_ta as ta
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import yfinance as yf

# Function to check if the US market is open
def is_us_market_open():
    """Checks if the current time is within US market hours (9:30 AM - 4:00 PM ET) and not on weekends."""
    eastern = pytz.timezone('US/Eastern')
    now = datetime.now(eastern)
    if now.weekday() in [5, 6]:  # Saturday and Sunday
        return False
    market_open = now.replace(hour=9, minute=30, second=0, microsecond=0)
    market_close = now.replace(hour=16, minute=0, second=0, microsecond=0)
    return market_open <= now <= market_close

# if not is_us_market_open():
#     print("The US market is currently closed. Script execution halted.")
# else:
load_dotenv()

# Function to fetch S&P 500 tickers from Wikipedia
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = [row.find('td').text.strip() for row in table.find_all('tr')[1:]]
    return tickers

# Function to send a message via Telegram
def send_telegram_message(message):
    bot_token = os.getenv("TELEGRAM_BOT_TOKEN")
    chat_id = os.getenv("TELEGRAM_CHAT_ID")
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    data = {"chat_id": chat_id, "text": message}
    requests.post(url, data=data)

# Retrieve the S&P 500 tickers and save them to a CSV file
sp500_tickers = get_sp500_tickers()
os.makedirs('stockdata', exist_ok=True)
pd.DataFrame(sp500_tickers, columns=["Ticker"]).to_csv('stockdata/sp500_tickers.csv', index=False)

# Read tickers from CSV
tickers_df = pd.read_csv('stockdata/sp500_tickers.csv')
tickers = tickers_df['Ticker'].tolist()

# --- New Bulk Download Approach Using a Cached & Rate-Limited Session ---
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter

class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND * 5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

# Create a Tickers object with the custom session.
# yf.Tickers accepts a space-separated string of tickers.
tickers_str = " ".join(tickers)
dat = yf.Tickers(tickers_str, session=session)

# Download historical data for all tickers (1 month period, 1-day interval)
try:
    his_data = dat.history(period='1mo', interval='1d')
except Exception as e:
    print(f"Error downloading bulk data: {e}")
    his_data = None

screener_results = []

if his_data is not None:
    # Process data for each ticker individually
    for ticker in tickers:
        try:
            # Extract the ticker-specific data from the MultiIndex DataFrame
            df = his_data.xs(ticker, axis=1, level='Ticker')
            # Flatten the columns to a single level (Price types)
            df.columns = df.columns.get_level_values(0)
            
            # Round OHLC values to two decimals if available
            for col in ['Open', 'High', 'Low', 'Close']:
                if col in df.columns:
                    df[col] = df[col].round(2)

            # Calculate linear regression curves using a 25- and 50-day lookback
            df['reg1'] = ta.linreg(df['Close'], length=25)
            df['reg2'] = ta.linreg(df['Close'], length=50)

            # Generate buy and sell signals
            df['buy_signal'] = np.where((df['reg1'] > df['reg2']) & (df['reg1'].shift(1) <= df['reg2'].shift(1)), 1, 0)
            df['sell_signal'] = np.where((df['reg1'] < df['reg2']) & (df['reg1'].shift(1) >= df['reg2'].shift(1)), 1, 0)

            # Save each ticker's data to a CSV file
            csv_filename = f'stockdata/{ticker}_1d_data.csv'
            df.to_csv(csv_filename)

            # Filter for recent data (last day)
            if not df.empty:
                recent_date_cutoff = df.index.max() - pd.Timedelta(days=1)
                df_recent = df[df.index >= recent_date_cutoff]
                df_filtered = df_recent[(df_recent['buy_signal'] == 1) | (df_recent['sell_signal'] == 1)]
                if not df_filtered.empty:
                    for index, row in df_filtered.iterrows():
                        screener_results.append({
                            'Ticker': ticker,
                            'Date': index,
                            'Buy Signal': row['buy_signal'],
                            'Sell Signal': row['sell_signal']
                        })
        except Exception as e:
            print(f"Failed to process data for ticker {ticker}: {e}")

# Save the screener results to a CSV file
screener_df = pd.DataFrame(screener_results)
screener_df.to_csv('Regression_cross_screener_results_1d.csv', index=False)

# Send the results via Telegram if there are any signals
if not screener_df.empty:
    message = (
        "Daily Screener Results:\n"
        "Buy = linreg 25 crossover linreg 50\n"
        "Sell = linreg 25 cross below linreg 50\n"
        f"{screener_df.to_string(index=False)}"
    )
    # send_telegram_message(message)

# Display a sample of the screener results
print("Daily Screener Results:\nBuy = linreg 25 crossover linreg 50\nSell = linreg 25 cross below linreg 50\n")
print(screener_df.tail())


KeyboardInterrupt: 

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s: %(message)s")


2025-02-23 18:02:54,936 INFO: Retrieved 503 tickers, sample: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']


In [10]:
logging.info("Retrieved %d tickers, sample: %s", len(sp500_tickers), sp500_tickers)


2025-02-23 18:03:26,250 INFO: Retrieved 503 tickers, sample: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK.B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BWA', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 

In [11]:
%time sp500_tickers = get_sp500_tickers()


CPU times: total: 891 ms
Wall time: 1.49 s


In [ ]:
%time
try:
    his_data = dat.history(period='1mo', interval='1d')
    print("Historical data shape:", his_data.shape)
except Exception as e:
    print("Error downloading bulk data:", e)


CPU times: total: 0 ns
Wall time: 0 ns


KeyboardInterrupt: 

[***********************************************208%*************************************************]  1045 of 503 completed

In [16]:
import os
import time
import pytz
import requests
import pandas as pd
import numpy as np
import pandas_ta as ta
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import yfinance as yf

# Check if US market is open
def is_us_market_open():
    """Checks if the current time is within US market hours (9:30 AM - 4:00 PM ET) and not on weekends."""
    eastern = pytz.timezone('US/Eastern')
    now = datetime.now(eastern)
    if now.weekday() in [5, 6]:  # Saturday and Sunday
        return False
    market_open = now.replace(hour=9, minute=30, second=0, microsecond=0)
    market_close = now.replace(hour=16, minute=0, second=0, microsecond=0)
    return market_open <= now <= market_close

# if not is_us_market_open():
#     print("The US market is currently closed. Script execution halted.")
# else:
load_dotenv()

# Function to fetch S&P 500 tickers from Wikipedia
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = [row.find('td').text.strip() for row in table.find_all('tr')[1:]]
    return tickers

# Function to send message to Telegram
def send_telegram_message(message):
    bot_token = os.getenv("TELEGRAM_BOT_TOKEN")
    chat_id = os.getenv("TELEGRAM_CHAT_ID")
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    data = {"chat_id": chat_id, "text": message}
    requests.post(url, data=data)

# Get the S&P 500 tickers and save to CSV
sp500_tickers = get_sp500_tickers()
os.makedirs('stockdata', exist_ok=True)
pd.DataFrame(sp500_tickers, columns=["Ticker"]).to_csv('stockdata/sp500_tickers.csv', index=False)

# Read tickers from CSV
tickers_df = pd.read_csv('stockdata/sp500_tickers.csv')
tickers = tickers_df['Ticker'].tolist()

# Define the timeframes (here using only daily data for 90 days)
timeframes = {
    '1d': timedelta(days=90),
    # Additional intervals (e.g., '15m', '1h', '1wk') can be added as needed
}
recent_period = 1  # in days

# --- New download approach using CachedLimiterSession and yf.Tickers ---
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter

class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND * 5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

# Create a Tickers object using all tickers with the custom session.
# Note: yf.Tickers accepts a string of tickers separated by spaces.
tickers_str = " ".join(tickers)
dat = yf.Tickers(tickers_str, session=session)

# Dictionary to hold downloaded data for each timeframe
data = {}
screener_results = []  # list to store screener signal results

# Loop over timeframes and download bulk historical data
for tf, delta in timeframes.items():
    # Use period string based on the desired date range, e.g. "90d" for 90 days
    period_str = f"{delta.days}d"
    try:
        his_data = dat.history(period=period_str, interval=tf)
    except Exception as e:
        print(f"Error downloading bulk data for interval {tf}: {e}")
        continue

    data[tf] = {}
    # his_data has a MultiIndex for columns: (Price, Ticker)
    for ticker in tickers:
        try:
            # Extract data for a single ticker from the MultiIndex DataFrame
            df = his_data.xs(ticker, axis=1, level='Ticker')
            # Flatten the columns (Price types) if needed
            df.columns = df.columns.get_level_values(0)
            # Round OHLC values to two decimals
            for col in ['Open', 'High', 'Low', 'Close']:
                if col in df.columns:
                    df[col] = df[col].round(2)

            # Calculate linear regression curves using pandas_ta
            df['reg1'] = ta.linreg(df['Close'], length=25)
            df['reg2'] = ta.linreg(df['Close'], length=50)

            # Generate buy and sell signals
            df['buy_signal'] = np.where((df['reg1'] > df['reg2']) & (df['reg1'].shift(1) <= df['reg2'].shift(1)), 1, 0)
            df['sell_signal'] = np.where((df['reg1'] < df['reg2']) & (df['reg1'].shift(1) >= df['reg2'].shift(1)), 1, 0)

            # Save each ticker's data to CSV
            csv_filename = f'stockdata/{ticker}_{tf}_data.csv'
            df.to_csv(csv_filename)

            # Filter for the recent period
            if not df.empty:
                recent_date_cutoff = df.index.max() - pd.Timedelta(days=recent_period)
                df_recent = df[df.index >= recent_date_cutoff]
                df_filtered = df_recent[(df_recent['buy_signal'] == 1) | (df_recent['sell_signal'] == 1)]
                if not df_filtered.empty:
                    for index, row in df_filtered.iterrows():
                        screener_results.append({
                            'Ticker': ticker,
                            'Date': index,
                            'Buy Signal': row['buy_signal'],
                            'Sell Signal': row['sell_signal']
                        })
        except Exception as e:
            print(f"Failed to process data for ticker {ticker} on timeframe {tf}: {e}")

# Save screener results to a CSV file
screener_df = pd.DataFrame(screener_results)
screener_df.to_csv('Regression_cross_screener_results_1d.csv', index=False)

# Send results to Telegram if available
if not screener_df.empty:
    message = (
        "Daily screener Results:\n"
        "Buy = linreg 25 crossover linreg 50\n"
        "Sell = linreg 25 cross below linreg 50\n"
        f"{screener_df.to_string(index=False)}"
    )
    # send_telegram_message(message)

# Display a sample of the screener results
print("Daily screener Results:\nBuy = linreg 25 crossover linreg 50\nSell = linreg 25 cross below linreg 50\n")
print(screener_df.tail())


[**********************81%**************         ]  405 of 503 completed2025-02-23 19:07:30,307 ERROR: Unable to deserialize response: a bytes-like object is required, not 'NoneType'
[*********************100%***********************]  502 of 503 completed
2025-02-23 19:07:36,892 ERROR: 
4 Failed downloads:
[*********************100%***********************]  502 of 503 completed2025-02-23 19:07:36,896 ERROR: ['BRK.B']: YFPricesMissingError('possibly delisted; no price data found  (period=90d) (Yahoo error = "No data found, symbol may be delisted")')
2025-02-23 19:07:36,897 ERROR: ['BF.B', 'ETR']: YFPricesMissingError('possibly delisted; no price data found  (period=90d)')
2025-02-23 19:07:36,898 ERROR: ['AXON']: InterfaceError('bad parameter or other API misuse')
C:\Users\sly\AppData\Local\Temp\ipykernel_10024\798355204.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

Daily screener Results:
Buy = linreg 25 crossover linreg 50
Sell = linreg 25 cross below linreg 50

   Ticker       Date  Buy Signal  Sell Signal
56    URI 2025-02-20         0.0          1.0
57    VMC 2025-02-21         0.0          1.0
58    WFC 2025-02-20         0.0          1.0
59     WY 2025-02-20         0.0          1.0
60    ZTS 2025-02-20         0.0          1.0


# fix version

In [17]:
import os
import time
import pytz
import requests
import pandas as pd
import numpy as np
import pandas_ta as ta
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import yfinance as yf

# Check if US market is open
def is_us_market_open():
    """Checks if the current time is within US market hours (9:30 AM - 4:00 PM ET) and not on weekends."""
    eastern = pytz.timezone('US/Eastern')
    now = datetime.now(eastern)
    if now.weekday() in [5, 6]:  # Saturday and Sunday
        return False
    market_open = now.replace(hour=9, minute=30, second=0, microsecond=0)
    market_close = now.replace(hour=16, minute=0, second=0, microsecond=0)
    return market_open <= now <= market_close

# For debugging in Jupyter, you can disable the market check.
# if not is_us_market_open():
#     print("The US market is currently closed. Script execution halted.")
# else:
load_dotenv()

# Function to fetch S&P 500 tickers from Wikipedia
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = [row.find('td').text.strip() for row in table.find_all('tr')[1:]]
    return tickers

# Function to send a message via Telegram
def send_telegram_message(message):
    bot_token = os.getenv("TELEGRAM_BOT_TOKEN")
    chat_id = os.getenv("TELEGRAM_CHAT_ID")
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    data = {"chat_id": chat_id, "text": message}
    requests.post(url, data=data)

# Get the S&P 500 tickers and save to CSV
sp500_tickers = get_sp500_tickers()
os.makedirs('stockdata', exist_ok=True)
pd.DataFrame(sp500_tickers, columns=["Ticker"]).to_csv('stockdata/sp500_tickers.csv', index=False)

# Read tickers from CSV
tickers_df = pd.read_csv('stockdata/sp500_tickers.csv')
tickers = tickers_df['Ticker'].tolist()

# Define the timeframes (here using only daily data for 90 days)
timeframes = {
    '1d': timedelta(days=90),
}
recent_period = 1  # in days

# --- New download approach using CachedLimiterSession and yf.Tickers ---
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from pyrate_limiter import Duration, RequestRate, Limiter

class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND * 5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
)

# Create a Tickers object using all tickers with the custom session.
# Note: yf.Tickers accepts a string of tickers separated by spaces.
tickers_str = " ".join(tickers)
dat = yf.Tickers(tickers_str, session=session)

data = {}
screener_results = []  # list to store screener signal results

for tf, delta in timeframes.items():
    # Use period string based on the desired date range, e.g. "90d" for 90 days
    period_str = f"{delta.days}d"
    try:
        his_data = dat.history(period=period_str, interval=tf)
    except Exception as e:
        print(f"Error downloading bulk data for interval {tf}: {e}")
        continue

    data[tf] = {}
    # Get available tickers in the downloaded data from the MultiIndex level 'Ticker'
    available_tickers = set(his_data.columns.get_level_values('Ticker'))
    for ticker in tickers:
        if ticker not in available_tickers:
            print(f"Ticker {ticker} not found in historical data, skipping.")
            continue
        try:
            # Extract data for a single ticker from the MultiIndex DataFrame and make a copy
            df = his_data.xs(ticker, axis=1, level='Ticker').copy()
            # Flatten the columns (Price types) if needed
            df.columns = df.columns.get_level_values(0)
            # Round OHLC values to two decimals using .loc to avoid SettingWithCopyWarning
            for col in ['Open', 'High', 'Low', 'Close']:
                if col in df.columns:
                    df.loc[:, col] = df[col].round(2)

            # Calculate linear regression curves using pandas_ta
            df['reg1'] = ta.linreg(df['Close'], length=25)
            df['reg2'] = ta.linreg(df['Close'], length=50)

            # Generate buy and sell signals
            df['buy_signal'] = np.where((df['reg1'] > df['reg2']) & (df['reg1'].shift(1) <= df['reg2'].shift(1)), 1, 0)
            df['sell_signal'] = np.where((df['reg1'] < df['reg2']) & (df['reg1'].shift(1) >= df['reg2'].shift(1)), 1, 0)

            # Save each ticker's data to a CSV file
            csv_filename = f'stockdata/{ticker}_{tf}_data.csv'
            df.to_csv(csv_filename)

            # Filter for the recent period
            if not df.empty:
                recent_date_cutoff = df.index.max() - pd.Timedelta(days=recent_period)
                df_recent = df[df.index >= recent_date_cutoff]
                df_filtered = df_recent[(df_recent['buy_signal'] == 1) | (df_recent['sell_signal'] == 1)]
                if not df_filtered.empty:
                    for index, row in df_filtered.iterrows():
                        screener_results.append({
                            'Ticker': ticker,
                            'Date': index,
                            'Buy Signal': row['buy_signal'],
                            'Sell Signal': row['sell_signal']
                        })
        except Exception as e:
            print(f"Failed to process data for ticker {ticker} on timeframe {tf}: {e}")

# Save the screener results to a CSV file
screener_df = pd.DataFrame(screener_results)
screener_df.to_csv('Regression_cross_screener_results_1d.csv', index=False)

# Optionally, send results to Telegram if available
if not screener_df.empty:
    message = (
        "Daily Screener Results:\n"
        "Buy = linreg 25 crossover linreg 50\n"
        "Sell = linreg 25 cross below linreg 50\n"
        f"{screener_df.to_string(index=False)}"
    )
    # Uncomment the following line to send the message:
    # send_telegram_message(message)

# Display a sample of the screener results
print("Daily Screener Results:\nBuy = linreg 25 crossover linreg 50\nSell = linreg 25 cross below linreg 50\n")
print(screener_df.tail())


[**********************90%******************     ]  451 of 503 completed2025-02-23 19:11:38,315 ERROR: Unable to deserialize response: a bytes-like object is required, not 'NoneType'
[*********************100%***********************]  503 of 503 completed
2025-02-23 19:11:44,328 ERROR: 
5 Failed downloads:
2025-02-23 19:11:44,331 ERROR: ['ETR', 'BF.B']: YFPricesMissingError('possibly delisted; no price data found  (period=90d)')
2025-02-23 19:11:44,332 ERROR: ['EIX', 'CHD']: InterfaceError('bad parameter or other API misuse')
2025-02-23 19:11:44,334 ERROR: ['BRK.B']: YFPricesMissingError('possibly delisted; no price data found  (period=90d) (Yahoo error = "No data found, symbol may be delisted")')


Daily Screener Results:
Buy = linreg 25 crossover linreg 50
Sell = linreg 25 cross below linreg 50

   Ticker       Date  Buy Signal  Sell Signal
55    URI 2025-02-20         0.0          1.0
56    VMC 2025-02-21         0.0          1.0
57    WFC 2025-02-20         0.0          1.0
58     WY 2025-02-20         0.0          1.0
59    ZTS 2025-02-20         0.0          1.0
